In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from evaluate_data import read_df, get_spec

#exp_name = '2020_10_30_dynamic' 
exp_name = '2020_10_30_dynamic_test' 
#exp_name = '2020_10_30_dynamic_move' 

df, df_pos = read_df(degree=90, props=False, snr=False, motors=False, source='mono', 
                     exp_name=exp_name)
df.head()
f_wav, t_wav, spectrogram_wav = get_spec(degree=90, props=False, snr=False, motors=False, source='mono', 
                     exp_name=exp_name)

In [ ]:
df_pos.head()

In [ ]:
n_columns = len(df)
mic_positions = df.iloc[0].mic_positions # n_mics x 2
frequencies = df.iloc[0].frequencies
print(frequencies)
times = df.audio_timestamp.values / 1e6
times -= times[0]

spectrogram = np.zeros((len(frequencies), n_columns))
i_col = 0
for __, row in df.iterrows():
    signals_f = row.signals_f
    spectrogram[:, i_col] = np.sum(np.abs(signals_f), axis=0)
    i_col += 1
    
plt.figure()
plt.pcolormesh(t_wav, f_wav, np.angle(spectrogram_wav))
plt.xlabel('time [s]')
plt.ylabel('angle [Hz]')
plt.title('measurement mic spectrogram')
    
plt.figure()
plt.pcolormesh(t_wav, f_wav, np.log10(np.abs(spectrogram_wav)))
plt.xlabel('time [s]')
plt.ylabel('frequency [Hz]')
plt.title('measurement mic spectrogram')

plt.figure()
plt.pcolormesh(times, frequencies, np.log10(spectrogram))
plt.xlabel('time [s]')
plt.ylabel('frequency [Hz]')
plt.title('recorded spectrogram')

#selected_idx = range(len(frequencies))
selected_hz = [800]
selected_idx = [np.argmin(np.abs(frequencies - hz)) for hz in selected_hz]
print(selected_idx)

plt.figure()
for i in selected_idx:
    plt.plot(times, spectrogram[i, :], label=f'{frequencies[i]:.1f}Hz')
plt.legend()

plt.figure()
mic_positions = df.iloc[0].mic_positions # n_mics x 2
for i in range(mic_positions.shape[0]):
    plt.scatter(mic_positions[i, 0], mic_positions[i, 1], label=f'mic{i}')
plt.legend()

In [ ]:
# resample df_pos at the timestamps in df
MAX_ALLOWED_LAG_MS = 20

for i, row in df.iterrows():
    timestamp = row.timestamp
    
    # most recent position timestamp
    pos_idx = df_pos[df_pos.timestamp < timestamp].index[-1]
    lag = timestamp - df_pos.loc[pos_idx].timestamp
    if lag <= MAX_ALLOWED_LAG_MS:
        df.loc[i, 'yaw_deg'] = df_pos.loc[pos_idx].yaw_deg
        df.loc[i, 'dx'] = df_pos.loc[pos_idx].dx
        df.loc[i, 'dy'] = df_pos.loc[pos_idx].dy
    else:
        df.loc[i, 'yaw_deg'] = None
        print('too high time lag (in ms):', lag)

In [ ]:
start_pos = np.array([0, 0])
positions = np.empty([len(df), 2])

plt.figure()
i_row = 0
for i, row in df.iterrows():
    yaw_rad = row.yaw_deg / 180 * np.pi
    length = np.sqrt(row.dx**2 + row.dy**2) 
    new_pos = start_pos + length * np.array([np.cos(yaw_rad), np.sin(yaw_rad)])
    positions[i_row, :] = new_pos
    i_row += 1
    plt.scatter(row.timestamp/1000-df.timestamp.iloc[0]/1000, row.yaw_deg, color='C0')
plt.xlabel('time [s]')
plt.ylabel('yaw [deg]')
    
plt.figure()
plt.scatter(positions[:, 0], positions[:, 1])
plt.axis('equal')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

In [ ]:
from audio_stack.beam_former import BeamFormer
from audio_stack.spectrum_estimator import normalize_rows, combine_rows

MAX_COUNTER = 10

combination_n = MAX_COUNTER
combination_method = "product"
normalization_method = "none"

frequencies = df.iloc[0].frequencies[selected_idx]

beam_former = BeamFormer(mic_positions=mic_positions)
beam_former.init_dynamic_estimate(combination_n, combination_method, normalization_method)
beam_former.init_multi_estimate(frequencies)

n_columns = len(df) #100
angles = beam_former.theta_scan
times = np.empty(n_columns)
raw_heatmap = np.empty((len(angles), n_columns))
dynamic_heatmap = np.empty((len(angles), n_columns))
multi_heatmap = np.empty((len(angles), n_columns))

multi_counter = 0
i_col = 0

print('start at degrees:', df.yaw_deg.iloc[0])
for __, row in df.iterrows():
    signals_f = row.signals_f[:, selected_idx]
    freqs = row.frequencies[selected_idx]
    
    if pd.isna(row.yaw_deg):
        print('skipping')
        continue
    
    orientation_deg = row.yaw_deg
    timestamp = row.timestamp
    
    R = beam_former.get_correlation(signals_f.T) # n_freqs x n_mics x n_mics
    #raw_spectrum = beam_former.get_das_spectrum(R, freqs)
    raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs)
    
    #### DYNAMIC
    beam_former.add_to_dynamic_estimates(raw_spectrum, orientation_deg=-orientation_deg)
    dynamic_spectrum = beam_former.get_dynamic_estimate()
    dynamic_spectrum = combine_rows(dynamic_spectrum, combination_method, keepdims=True)
    dynamic_spectrum = normalize_rows(dynamic_spectrum, method="zero_to_one")
    #dynamic_spectrum = beam_former.shift_spectrum(dynamic_spectrum, orientation_deg)
    
    #### RAW
    raw_spectrum = combine_rows(raw_spectrum, combination_method, keepdims=True)
    raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")
    
    #### MULTI
    time_sec = row.audio_timestamp / 1e6
    beam_former.add_to_multi_estimate(signals_f.T, freqs, time_sec, -orientation_deg)
    multi_counter += 1 #TODO(FD) add this to beam_former
    multi_spectrum = beam_former.get_multi_estimate(method='mvdr')
    multi_spectrum = combine_rows(multi_spectrum, combination_method, keepdims=True)
    multi_spectrum = normalize_rows(multi_spectrum, method="zero_to_one")
    
    if multi_counter > MAX_COUNTER:
        beam_former.init_multi_estimate(frequencies)
        multi_counter = 0
    
    raw_heatmap[:, i_col] = raw_spectrum.flatten()
    dynamic_heatmap[:, i_col] = dynamic_spectrum.flatten()
    multi_heatmap[:, i_col] = multi_spectrum.flatten()
    times[i_col] = time_sec
    i_col += 1
        
    if i_col >= n_columns:
        break

In [ ]:
print(multi_heatmap.shape)

times -= times[0]

fig, ax = plt.subplots()
ax.pcolormesh(times, angles, multi_heatmap)
ax.set_title('multi heatmap')
ax.set_xlabel('time [s]')
ax.set_ylabel('angle [rad]')

fig, ax = plt.subplots()
ax.pcolormesh(times, angles, dynamic_heatmap)
ax.set_title('dynamic heatmap')

fig, ax = plt.subplots()
ax.pcolormesh(times, angles, raw_heatmap)
ax.set_title('raw heatmap')